In [1]:
%load_ext autoreload
%autoreload 2

import torch
import matplotlib.pyplot as plt
from neurorient import NeurOrient

In [2]:
from mpi4py import MPI

In [3]:
spi_data = torch.load('/pscratch/sd/z/zhantao/neurorient_repo/data/1bxr_train.pt')
model_dir = '/pscratch/sd/z/zhantao/neurorient_repo/model'
print(spi_data.keys())

dict_keys(['orientations', 'images', 'angles', 'points', 'pixel_position_reciprocal', 'volume', 'img_real_mesh', 'vol_real_mesh'])


In [4]:
model = NeurOrient(spi_data['pixel_position_reciprocal'], path=model_dir)
model.to('cpu');

| Wide-Resnet 16x2
start building 1
layer 1 built
start building 2
layer 2 built
start building 3
layer 3 built
MODEL TOPOLOGY:
	0 - 
	1 - conv1
	2 - conv1._basisexpansion
	3 - conv1._basisexpansion.block_expansion_('irrep_0', 'regular')
	4 - layer1
	5 - layer1.0
	6 - layer1.0.bn1
	7 - layer1.0.bn1.batch_norm_[8]
	8 - layer1.0.relu1
	9 - layer1.0.conv1
	10 - layer1.0.conv1._basisexpansion
	11 - layer1.0.conv1._basisexpansion.block_expansion_('regular', 'regular')
	12 - layer1.0.bn2
	13 - layer1.0.bn2.batch_norm_[8]
	14 - layer1.0.relu2
	15 - layer1.0.dropout
	16 - layer1.0.conv2
	17 - layer1.0.conv2._basisexpansion
	18 - layer1.0.shortcut
	19 - layer1.0.shortcut._basisexpansion
	20 - layer1.0.shortcut._basisexpansion.block_expansion_('regular', 'regular')
	21 - layer1.1
	22 - layer1.1.bn1
	23 - layer1.1.bn1.batch_norm_[8]
	24 - layer1.1.relu1
	25 - layer1.1.conv1
	26 - layer1.1.bn2
	27 - layer1.1.bn2.batch_norm_[8]
	28 - layer1.1.relu2
	29 - layer1.1.dropout
	30 - layer1.1.conv2
	31 - 

In [5]:
# model.compute_autocorrelation()
# orientations = model.image_to_orientation(spi_data['images'][:10].unsqueeze(1))
# from neurorient.reconstruction.slicing import get_rho_function
# rho_true = torch.fft.fftshift(torch.fft.ifftn(torch.fft.ifftshift(spi_data['volume']))).real.clip(0)
# rho_func = get_rho_function(spi_data['vol_real_mesh'], rho_true)
# rho_img_coords = torch.from_numpy(rho_func(spi_data['img_real_mesh'].numpy()))
# rho_true = torch.from_numpy(rho_func(model.grid_position_real.numpy()))
# ac = model.compute_autocorrelation(rho=rho_true)
# slices = model.gen_slices(ac, orientations)
# plt.imshow(slices.detach().numpy()[0].clip(0), vmax=slices.max() * 0.001)

In [6]:
import lightning as L

In [7]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [8]:
dataset = TensorDataset(spi_data['images'].unsqueeze(1))
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

In [9]:
# batch = next(iter(dataloader))
# model.training_step(batch, 0)

In [10]:
from neurorient.utils_visualization import display_images_in_parallel, display_volumes

# display_images_in_parallel(torch.randn(10, 3,3), torch.randn(10, 3,3))
display_volumes(torch.randn(10, 3,3,3))

In [11]:
trainer = L.Trainer(
    max_epochs=10, accelerator='cpu',
    log_every_n_steps=1, devices=torch.cuda.device_count(),
    enable_checkpointing=True, default_root_dir=model.path)
trainer.fit(model, dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                  | Type     | Params
---------------------------------------------------
0 | orientation_predictor | I2S      | 2.1 M 
1 | rho_predictor         | SirenNet | 264 K 
2 | nufft_forward         | KbNufft  | 0     
---------------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.338     Total estimated model params size (MB)


Epoch 0:   0%|          | 5/1000 [03:10<10:30:30, 38.02s/it, v_num=12]